# ROBin's Training

What this notebook is and not:
* This notebook is not the model that ROBin uses.
* This notebook is only the orchestrator to training ROBin's material and items model.

Due to limitation to resources, ROBin's training has only been limited to google's Colab.

## Helpers for mounting Gdrive and fetch source from repo

In [ ]:
import requests
from io import BytesIO
from zipfile import ZipFile
import re
import os
from shutil import rmtree, copyfile
from google.colab import drive

### Mirroring the main repo
Since the companies repositories are all private and therefore Colab has no access to it, another public repo is setup on a personal github account to mirror the repository.

In [ ]:
def download_robin_source(git_url):
  r = requests.get(git_url)
  mem_zip = BytesIO(r.content)

  root_dir=None
  src_dir=None

  with ZipFile(mem_zip, 'r') as zipObj:
    pathnames = zipObj.namelist()
    idx = list(map(lambda path: bool(re.search("src/$", path)), pathnames)).index(True)
    root_dir = pathnames[0]
    src_dir = pathnames[idx]
    zipObj.extractall()

  os.rename(src_dir, 'src')
  rmtree(root_dir)

### Fetching dataset stored on google drive

In [ ]:
from typing import List, Iterable

def fetch_from_gdrive(list_of_zips: List[Iterable]):
  drive.mount('drive')

  for zip, gdrive_path in list_of_zips:
    if not os.path.exists(zip):
      print("Copying", zip)
      copyfile(src=gdrive_path, dst=zip)
    else: 
      print(zip, "already exists")
  
  drive.flush_and_unmount() # Unmounting Google Drive as a good practice

In [ ]:
repo_mirror_url = # 'https://github.com/<git_username>/<mirror_repo>/archive/<branch>.zip'
download_robin_source(repo_mirror_url)

## Setting up project structure on Colab

In [ ]:
import os
from datetime import datetime
from pathlib import Path
from src.model_builder import RobinMobilenetV2
from src.helpers import create_dir_if_not_exist, split_train_valid_test,\
  get_dataset_distribution
from src.pipelines import create_input_pipelines, \
    train_new_model, fine_tune_model, evaluate_model
from src.log_utils import plot_class_distribution, log_as_image

In [ ]:

data_dir=Path(os.getcwd())/'data'

images_dir= data_dir/'images'
train_dir= data_dir/'train'
valid_dir= data_dir/'valid'
test_dir= data_dir/'test'

model_dir = Path(os.getcwd())/'models'
save_model_dir = model_dir/'saved'
logs_dir = Path(os.getcwd())/'logs'
export_dir = Path(os.getcwd())/'export'

project_dir=[
  images_dir,
  train_dir,
  valid_dir,
  test_dir,
  save_model_dir,
  lite_models_dir,
  logs_dir,
  export_dir
]

create_dir_if_not_exist(project_dir)

## Fetch data from drive

In [ ]:
zipped_dataset_name = # Name of zipped images eg. 'materials_ds_v2.zip'
gdrive_path_to_dataset = # eg. path to zipped data f"/content/drive/My Drive/Colab Notebooks/ROBin/datasets/{zipped_dataset_name}"

fetch_from_gdrive([(zipped_dataset_name, gdrive_path_to_dataset)])

# Some boilerplate to unzip and remove unnecessary files from project structure
zip_path = # update
zip_to_dest = # update

ZipFile(zip_path).extractall(zip_to_dest)
os.remove(zip_path)

## Split data randomly

By setting `train_ratio` to 0.7 and `valid_ratio` to 0.15, that leaves a the `test_ratio` to 0.15 as well

In [ ]:
split_train_valid_test(
    images_dir,
    train_dir,
    valid_dir,
    test_dir,
    train_ratio=0.7,
    valid_ratio=0.15
)

## Visualise class distribution 

Visualising at this point prior to training allows for the scientist to decide if he/she would like to up/down-sample.

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image

train_distro = get_dataset_distribution(train_dir)
valid_distro = get_dataset_distribution(valid_dir)
test_distro = get_dataset_distribution(test_dir)
class_names = list(train_distro.keys())

distro_plot = plot_class_distribution(
    class_names,
    [ train_distro[key] for key in train_distro ],
    [ valid_distro[key] for key in valid_distro ],
    [ test_distro[key] for key in test_distro ],
)

plt.axis('off')
plt.imshow(Image.open(distro_plot))

In [ ]:
plt.close() # Closing the figure so that it doesn't affect the plots to be logged for Tensorboard

## Training

### Declare the variation the hyperparameters to train with

In [ ]:
hyperparams_01 = {
    "img_size": (224, 224),
    "channels": 3,
    "batch_size": 16,
    "base_learning_rate": 0.0001,
    "initial_epochs": 15,
    "fine_tune_from": 100,
    "fine_tune_epochs": 10
}

hyperparams_02 = {
    "img_size": (224, 224),
    "channels": 3,
    "batch_size": 32,
    "base_learning_rate": 0.0001,
    "initial_epochs": 20,
    "fine_tune_from": 100,
    "fine_tune_epochs": 15
}

hyperparams_03 = {
    "img_size": (224, 224),
    "channels": 3,
    "batch_size": 32,
    "base_learning_rate": 0.0005,
    "initial_epochs": 25,
    "fine_tune_from": 100,
    "fine_tune_epochs": 20
}

### Define the training pipeline

In this case, we create a brand new model for each training.

In [ ]:
def start_training_new_model(model_builder, hyperparams, training_tag, datasets, logdir, save_to):
    # Log Class Distribution
    log_as_image(
        os.path.join(logdir,'class_distribution'),
        "Class Distribution Plot",
        distro_plot
    )

    """
    IMPORTANT!!!
    The model instance generated from train_new_model is stateful and its state
    is updated across the different pipelines
    """

    # Creates a new model by transfering learning and trains its new classifer layers
    (robin_model, history) = train_new_model(
    model_builder=model_builder,
    hyperparams=hyperparams,
    datasets=datasets,
    log_dir=logdir,
    training_tag=training_tag
    )

    evaluate_model(
        model=robin_model(),
        class_names=datasets['class_names'],
        test_ds=datasets["test_ds"],
        log_dir=logdir,
        training_tag=training_tag,
        cm_name="New Classifier for Materials Model",
        log_false_images=False
    )

    # Unlocks several layers before the classifer and further trains the model
    fine_tune_model(
        model_instance=robin_model,
        hyperparams=hyperparams,
        datasets=datasets,
        log_dir=logdir,
        training_tag=training_tag,
        history=history
    )

    evaluate_model(
        model=robin_model(),
        class_names=datasets['class_names'],
        test_ds=datasets["test_ds"],
        log_dir=logdir,
        training_tag=training_tag,
        cm_name="Fine Tune Materials Model",
        log_false_images=True
    )
    tf.saved_model.save(robin_model(), save_to)

    # Zipping for exporting to external/cloud storage
    zip_dir(f'/content/logs/{training_tag}', dst_dir="export/logs")
    zip_dir(f'/content/models/{training_tag}', dst_dir="export/models")

### Perform training with the variations of hyerparameters

In [ ]:

for hyperparams_variation in [hyperparams_01, hyperparams_02, hyperparams_03]:

  train_ds, validation_ds, test_ds = create_input_pipelines(
      [train_dir, valid_dir, test_dir],
      hyperparams_variation
  )

  dataset = {
      "train_ds": train_ds,
      "validate_ds": validation_ds,
      "test_ds": test_ds,
      "class_names": train_ds.class_names
  }

  model_context= #update  "material" or "items"
  training_tag = f"{datetime.now().strftime('%Y%m%d-%H%M')}_{model_context}"

  save_path = os.path.join(save_model_dir, training_tag)
  log_path = os.path.join(logdir, training_tag)

  start_training_new_model(
      model_builder=RobinMobilenetV2,
      hyperparams=hyperparams_variation,
      training_tag=training_tag,
      datasets=dataset,
      logdir=log_path,
      save_to = save_path
  )